In [0]:
movies_df = spark.table("az_adb_simbus_training.imdb_project_schema.bronze_title_basics")
movies_df = (movies_df.select('tconst','titleType','primaryTitle','startYear').filter('titleType = "movie"')).na.drop(how='any')
#display(movies_df)


In [0]:
actor_or_director_df = spark.table("az_adb_simbus_training.imdb_project_schema.bronze_title_principals")
actor_or_director_df = (actor_or_director_df.select('tconst','category','nconst').filter('category = "actor" or category = "director"')).na.drop(how='any')
#display(actor_or_director_df)

In [0]:
join_df = movies_df.join(actor_or_director_df, movies_df.tconst == actor_or_director_df.tconst, how="inner")
join_df = join_df.drop(actor_or_director_df["tconst"],'titleType')
#display(join_df)

In [0]:
from pyspark.sql.functions import to_date, col, lit

names_act_direct = spark.table("az_adb_simbus_training.imdb_project_schema.bronze_name_basics")

#fetch data of people who are born in the 21st century
names_act_direct = (names_act_direct.select('nconst','primaryName').
filter(to_date(col("birthYear"), 'yyyy-MM-dd') >= to_date(lit('2001-01-01'), 'yyyy-MM-dd'))).na.drop(how='any')
#display(names_act_direct)

In [0]:
movie_rating_df = spark.table("az_adb_simbus_training.imdb_project_schema.bronze_title_ratings")
movie_rating_df = (movie_rating_df.select('tconst','averageRating','numVotes')).na.drop(how='any')
#display(movie_rating_df)
rating_join_df = join_df.join(movie_rating_df, movie_rating_df.tconst == join_df.tconst, how="inner")
rating_join_df=rating_join_df.drop('tconst')
#display(rating_join_df)


# final df created from joining the rating join with the names_act_direct dataframe
final_df_with_names=names_act_direct.join(rating_join_df, names_act_direct.nconst == rating_join_df.nconst, how="inner").drop("nconst")
#display(final_df_with_names)

# changed the headers
new_columns = ["person_name", "movie_title",'start_Year',"role","average_rating","num_votes"]
final_df_with_names = final_df_with_names.toDF(*new_columns)
display(final_df_with_names)


In [0]:
#final_df_with_names=final_df_with_names.drop("region")


final_df_with_names.write.format("delta").mode("overwrite").saveAsTable("az_adb_simbus_training.imdb_project_schema.silver_popular_actor_directors")